# Notebook 02: Feature Analysis
## Feature Rationale & Diagnostic Validation

### Objectives
1. Justify the chosen feature set (link statistics to why features were retained)
2. Demonstrate feature engineering impact (pre vs. post)
3. Validate feature quality (distributions, correlations, vehicle-specific behavior)
4. Identify most predictive features for theft detection

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from src.config.loader import load_config
from src.features.engineering import FeatureEngineer
from src.utils.timezone import ensure_series_utc

config = load_config(
    detection_config_path=Path("../config/detection_config.yaml"),
    model_config_path=Path("../config/model_config.yaml"),
    path_config_path=Path("../config/paths_config.yaml")
)

# Load events (should have features already engineered from 03_train_models.py)
events_df = pd.read_csv(config.paths.output.events_csv)
events_df['start_time'] = pd.to_datetime(events_df['start_time'], utc=True)
events_df['end_time'] = pd.to_datetime(events_df['end_time'], utc=True)

# Load feature importance (from training)
fi_path = Path("../data/reports/feature_importance.csv")
if fi_path.exists():
    feature_importance = pd.read_csv(fi_path)
else:
    print("⚠️ Feature importance not found. Run 03_train_models.py first.")
    feature_importance = None

print(f"Loaded {len(events_df):,} events with {len([c for c in events_df.columns if c not in ['vehicle_id','start_time','end_time','pattern','y','is_train']])} features")

## Section 1: Feature Categories & Purpose (5 minutes)

### 1.1 Feature Inventory

In [ ]:
# Categorize features
feature_categories = {
    'Event Core': ['drop_gal', 'min_step_gal', 'duration_min', 'n_negative_steps', 'rate_gpm'],
    'Event Quality': ['n_points', 'median_dt_s', 'p95_abs_dfuel', 'pct_ign_on'],
    'Temporal': ['hod_sin', 'hod_cos', 'weekday', 'is_weekend', 'is_night'],
    'Spatial': ['lat_c', 'lon_c', 'lat_std', 'lon_std', 'coord_range_km', 'window_trip_km'],
    'Hotspot': ['is_hotspot', 'cluster_id', 'cluster_count'],
    'Behavioral': ['pre_event_distance_km', 'pre_event_avg_speed', 'pre_event_moving_pct', 
                   'pre_event_fuel_change', 'speed_std', 'speed_max', 'movement_variability'],
    'Vehicle-Normalized': ['drop_gal_vehicle_zscore', 'rate_gpm_vehicle_zscore', 
                           'min_step_gal_vehicle_zscore', 'duration_min_vehicle_zscore',
                           'drop_pct_of_avg', 'drop_pct_of_max']
}

# Count available features per category
for category, features in feature_categories.items():
    available = [f for f in features if f in events_df.columns]
    print(f"{category:20s}: {len(available)}/{len(features)} available")

**Design Principle:**
> Each feature category serves a specific purpose in the detection pipeline:
> - **Event Core**: Fundamental theft characteristics (volume, rate, duration)
> - **Event Quality**: Signal quality indicators (helps filter sensor noise)
> - **Temporal**: Time-of-day patterns (night thefts more suspicious)
> - **Spatial**: Location context (repeated locations = organized theft)
> - **Hotspot**: Geospatial clustering (high-risk zones)
> - **Behavioral**: Pre-event context (sudden stops after long drives = suspicious)
> - **Vehicle-Normalized**: Account for vehicle-specific baselines (fleet heterogeneity)

## Section 2: Core Features - Impact Demonstration (15 minutes)

### 2.1 drop_gal: Fuel Drop Volume
**Rationale:** Primary indicator of theft magnitude. Larger drops more likely to be theft vs. consumption.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Distribution
axes[0].hist(events_df['drop_gal'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(events_df['drop_gal'].median(), color='red', linestyle='--', 
                label=f'Median: {events_df["drop_gal"].median():.2f}gal')
axes[0].set_xlabel('Fuel Drop (gallons)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('drop_gal Distribution')
axes[0].legend()
axes[0].set_yscale('log')

# By pattern
events_df.boxplot(column='drop_gal', by='pattern', ax=axes[1])
axes[1].set_xlabel('Pattern')
axes[1].set_ylabel('Fuel Drop (gal)')
axes[1].set_title('drop_gal by Pattern')
plt.sca(axes[1])
plt.xticks(rotation=45)

# Theft vs. Normal (if labels available)
if 'y' in events_df.columns:
    theft_drops = events_df[events_df['y'] == 1]['drop_gal']
    normal_drops = events_df[events_df['y'] == 0]['drop_gal']
    
    axes[2].hist(normal_drops, bins=30, alpha=0.6, label='Normal', color='blue')
    axes[2].hist(theft_drops, bins=30, alpha=0.6, label='Theft', color='red')
    axes[2].set_xlabel('Fuel Drop (gal)')
    axes[2].set_ylabel('Frequency')
    axes[2].set_title('drop_gal: Theft vs. Normal')
    axes[2].legend()
    axes[2].set_yscale('log')
    
    # Statistical test
    stat, pval = stats.mannwhitneyu(theft_drops, normal_drops, alternative='greater')
    print(f"Mann-Whitney U test: statistic={stat:.2f}, p-value={pval:.2e}")
    print(f"Median drop - Theft: {theft_drops.median():.2f}gal, Normal: {normal_drops.median():.2f}gal")

plt.tight_layout()

### 2.2 rate_gpm: Fuel Drop Rate
**Rationale:** Distinguishes theft (rapid) from consumption (gradual). High rates impossible via normal operation.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Rate distribution with thresholds
axes[0].hist(events_df['rate_gpm'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(0.5, color='orange', linestyle='--', label='Typical consumption rate')
axes[0].axvline(2.0, color='red', linestyle='--', label='Max plausible rate (config)')
axes[0].set_xlabel('Rate (gal/min)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('rate_gpm Distribution')
axes[0].set_xlim(0, min(5, events_df['rate_gpm'].quantile(0.99)))
axes[0].legend()

# Scatterplot: drop vs. rate (colored by theft status)
if 'y' in events_df.columns:
    scatter = axes[1].scatter(events_df['drop_gal'], events_df['rate_gpm'], 
                             c=events_df['y'], cmap='RdYlGn_r', alpha=0.6, s=20)
    axes[1].set_xlabel('Fuel Drop (gal)')
    axes[1].set_ylabel('Rate (gal/min)')
    axes[1].set_title('Drop Volume vs. Rate (colored by theft label)')
    axes[1].set_ylim(0, min(3, events_df['rate_gpm'].quantile(0.99)))
    plt.colorbar(scatter, ax=axes[1], label='Theft')

plt.tight_layout()

**Key Finding:**
> Events with rate > 1.0 gal/min are [X]x more likely to be confirmed thefts (odds ratio).

## Section 3: Temporal Features - Night Effect (10 minutes)

### 3.1 Hour-of-Day Cyclical Encoding
**Rationale:** Thefts exhibit temporal patterns (e.g., overnight when supervision is low). Cyclical encoding (sin/cos) preserves 23→0 hour continuity.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calculate hour from midpoint
events_df['mid_time'] = events_df['start_time'] + (events_df['end_time'] - events_df['start_time'])/2
events_df['hour'] = events_df['mid_time'].dt.hour + events_df['mid_time'].dt.minute/60

# Hour distribution
axes[0].hist(events_df['hour'], bins=24, edgecolor='black', alpha=0.7, range=(0,24))
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Event Count')
axes[0].set_title('Event Distribution by Hour')
axes[0].axvspan(22, 24, alpha=0.2, color='navy', label='Night (22:00-05:59)')
axes[0].axvspan(0, 6, alpha=0.2, color='navy')
axes[0].legend()

# Theft rate by hour (if labels available)
if 'y' in events_df.columns:
    hour_bins = pd.cut(events_df['hour'], bins=24, labels=range(24))
    theft_rate_by_hour = events_df.groupby(hour_bins)['y'].mean()
    
    axes[1].bar(range(24), theft_rate_by_hour, edgecolor='black', alpha=0.7)
    axes[1].axvspan(22, 24, alpha=0.2, color='navy')
    axes[1].axvspan(0, 6, alpha=0.2, color='navy')
    axes[1].set_xlabel('Hour of Day')
    axes[1].set_ylabel('Theft Rate')
    axes[1].set_title('Confirmed Theft Rate by Hour')
    axes[1].set_ylim(0, 1)

plt.tight_layout()

# Statistical comparison: night vs. day
if 'y' in events_df.columns and 'is_night' in events_df.columns:
    night_theft_rate = events_df[events_df['is_night'] == 1]['y'].mean()
    day_theft_rate = events_df[events_df['is_night'] == 0]['y'].mean()
    print(f"Night theft rate: {night_theft_rate:.2%}")
    print(f"Day theft rate: {day_theft_rate:.2%}")
    print(f"Night/Day theft rate ratio: {night_theft_rate/day_theft_rate:.2f}x")

## Section 4: Behavioral Features - Pre-Event Context (15 minutes)

### 4.1 Pre-Event Distance (2-hour lookback)
**Rationale:** Thefts after long journeys may indicate planned stops at vulnerable locations. Post-journey exhaustion/inattention exploited by thieves.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Distribution
axes[0,0].hist(events_df['pre_event_distance_km'], bins=50, edgecolor='black', alpha=0.7)
axes[0,0].axvline(events_df['pre_event_distance_km'].median(), color='red', linestyle='--',
                  label=f'Median: {events_df["pre_event_distance_km"].median():.1f}km')
axes[0,0].set_xlabel('Pre-Event Distance (km, 2h lookback)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].set_title('Pre-Event Distance Distribution')
axes[0,0].legend()

# By pattern
events_df.boxplot(column='pre_event_distance_km', by='pattern', ax=axes[0,1])
axes[0,1].set_xlabel('Pattern')
axes[0,1].set_ylabel('Pre-Event Distance (km)')
axes[0,1].set_title('Pre-Event Distance by Pattern')
plt.sca(axes[0,1])
plt.xticks(rotation=45)

# Theft vs. Normal
if 'y' in events_df.columns:
    theft_dist = events_df[events_df['y'] == 1]['pre_event_distance_km']
    normal_dist = events_df[events_df['y'] == 0]['pre_event_distance_km']
    
    axes[1,0].hist(normal_dist, bins=30, alpha=0.6, label='Normal', color='blue')
    axes[1,0].hist(theft_dist, bins=30, alpha=0.6, label='Theft', color='red')
    axes[1,0].set_xlabel('Pre-Event Distance (km)')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Pre-Event Distance: Theft vs. Normal')
    axes[1,0].legend()
    
    # Statistical test
    stat, pval = stats.mannwhitneyu(theft_dist.dropna(), normal_dist.dropna())
    print(f"Pre-Event Distance - Mann-Whitney U: p={pval:.3f}")
    print(f"  Theft median: {theft_dist.median():.1f}km, Normal median: {normal_dist.median():.1f}km")

# Pre-event fuel change
axes[1,1].scatter(events_df['pre_event_distance_km'], events_df['pre_event_fuel_change'],
                 alpha=0.5, s=10)
axes[1,1].set_xlabel('Pre-Event Distance (km)')
axes[1,1].set_ylabel('Pre-Event Fuel Change (gal)')
axes[1,1].set_title('Pre-Event Distance vs. Fuel Change')
axes[1,1].axhline(0, color='red', linestyle='--', linewidth=1)

plt.tight_layout()

### 4.2 Movement Variability
**Rationale:** Sudden stops (low variability) vs. gradual slowdowns. Abrupt stops may indicate intentional parking for theft.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

if 'y' in events_df.columns:
    theft_var = events_df[events_df['y'] == 1]['movement_variability']
    normal_var = events_df[events_df['y'] == 0]['movement_variability']
    
    ax.hist(normal_var, bins=30, alpha=0.6, label='Normal', color='blue')
    ax.hist(theft_var, bins=30, alpha=0.6, label='Theft', color='red')
    ax.set_xlabel('Movement Variability (speed variance)')
    ax.set_ylabel('Frequency')
    ax.set_title('Movement Variability: Theft vs. Normal')
    ax.set_yscale('log')
    ax.legend()
    
    print(f"Movement Variability - Theft median: {theft_var.median():.2f}, Normal median: {normal_var.median():.2f}")

## Section 5: Vehicle-Normalized Features (10 minutes)

### 5.1 Why Vehicle Normalization Matters
**Problem:** Fleet heterogeneity (trucks vs. vans, different tank sizes). 5-gallon drop = minor for truck, major for van.

In [ ]:
# Example: drop_gal vs. drop_pct_of_avg for 3 vehicles
example_vids = events_df['vehicle_id'].value_counts().head(3).index

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for vid in example_vids:
    vid_events = events_df[events_df['vehicle_id'] == vid]
    axes[0].scatter(vid_events.index, vid_events['drop_gal'], label=f'Vehicle {vid}', alpha=0.7)
    axes[1].scatter(vid_events.index, vid_events['drop_pct_of_avg'], label=f'Vehicle {vid}', alpha=0.7)

axes[0].set_xlabel('Event Index')
axes[0].set_ylabel('Absolute Drop (gal)')
axes[0].set_title('Absolute Fuel Drop (not vehicle-normalized)')
axes[0].legend()

axes[1].set_xlabel('Event Index')
axes[1].set_ylabel('Drop as % of Vehicle Avg Fuel')
axes[1].set_title('Normalized Drop (% of vehicle average)')
axes[1].legend()

plt.tight_layout()

**Key Insight:**
> Without normalization, model biases toward vehicles with larger tanks. Normalization ensures equal treatment across fleet.

### 5.2 Z-Score Features

In [ ]:
# Correlation: raw vs. z-scored features
features_to_compare = [
    ('drop_gal', 'drop_gal_vehicle_zscore'),
    ('rate_gpm', 'rate_gpm_vehicle_zscore'),
    ('duration_min', 'duration_min_vehicle_zscore')
]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, (raw, zscore) in enumerate(features_to_compare):
    if zscore in events_df.columns:
        axes[i].scatter(events_df[raw], events_df[zscore], alpha=0.5, s=10)
        axes[i].set_xlabel(f'{raw} (raw)')
        axes[i].set_ylabel(f'{zscore}')
        axes[i].set_title(f'Raw vs. Vehicle Z-Score: {raw}')
        
        # Correlation
        corr = events_df[[raw, zscore]].corr().iloc[0,1]
        axes[i].text(0.05, 0.95, f'ρ = {corr:.3f}', 
                    transform=axes[i].transAxes, fontsize=12, verticalalignment='top')

plt.tight_layout()

## Section 6: Feature Importance from Trained Models (10 minutes)

### 6.1 Top 20 Features (from Random Forest)

In [ ]:
if feature_importance is not None:
    top_features = feature_importance.head(20)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(range(len(top_features)), top_features['importance'], color='steelblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'])
    ax.invert_yaxis()
    ax.set_xlabel('Feature Importance')
    ax.set_title('Top 20 Features - Random Forest')
    plt.tight_layout()

**Interpretation Guide:**

In [ ]:
if feature_importance is not None:
    # Categorize top features
    top_20_features = feature_importance.head(20)['feature'].tolist()
    
    categorized_top = {}
    for category, features in feature_categories.items():
        categorized_top[category] = [f for f in top_20_features if f in features]
    
    print("\nTop 20 Features by Category:")
    for category, features in categorized_top.items():
        if features:
            print(f"  {category}: {len(features)} features")
            for f in features:
                importance = feature_importance[feature_importance['feature'] == f]['importance'].values[0]
                print(f"    - {f}: {importance:.4f}")

### 6.2 Feature Correlation Matrix (Top Features)

In [ ]:
if feature_importance is not None:
    # Select top 15 features for correlation analysis
    top_15_features = feature_importance.head(15)['feature'].tolist()
    available_features = [f for f in top_15_features if f in events_df.columns]
    
    if available_features:
        corr_matrix = events_df[available_features].corr()
        
        fig, ax = plt.subplots(figsize=(12, 10))
        sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                   vmin=-1, vmax=1, center=0, ax=ax, square=True)
        ax.set_title('Feature Correlation Matrix (Top 15 Features)')
        plt.tight_layout()
        
        # Identify highly correlated pairs
        high_corr_pairs = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > 0.7:
                    high_corr_pairs.append((
                        corr_matrix.columns[i],
                        corr_matrix.columns[j],
                        corr_val
                    ))
        
        if high_corr_pairs:
            print("\n⚠️ Highly correlated feature pairs (|ρ| > 0.7):")
            for f1, f2, corr in high_corr_pairs:
                print(f"  {f1} ↔ {f2}: ρ = {corr:.3f}")

## Section 7: Feature Quality Diagnostics (10 minutes)

### 7.1 Missing Value Analysis

In [ ]:
numeric_features = [c for c in events_df.select_dtypes(include=[np.number]).columns 
                   if c not in ['vehicle_id', 'cluster_id', 'y', 'is_train']]

missing_pct = events_df[numeric_features].isnull().sum() / len(events_df) * 100
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)

if not missing_pct.empty:
    fig, ax = plt.subplots(figsize=(10, max(6, len(missing_pct)*0.3)))
    ax.barh(range(len(missing_pct)), missing_pct.values, color='coral')
    ax.set_yticks(range(len(missing_pct)))
    ax.set_yticklabels(missing_pct.index)
    ax.invert_yaxis()
    ax.set_xlabel('Missing Percentage (%)')
    ax.set_title('Features with Missing Values')
    ax.axvline(5, color='red', linestyle='--', label='5% threshold')
    ax.legend()
    plt.tight_layout()
    
    print(f"Features with >5% missing: {(missing_pct > 5).sum()}")
else:
    print("✅ No missing values in numeric features")


### 7.2 Outlier Detection (IQR Method)

In [ ]:
# Check for extreme outliers in key features
key_features = ['drop_gal', 'rate_gpm', 'duration_min', 'pre_event_distance_km']
available_key = [f for f in key_features if f in events_df.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, feature in enumerate(available_key):
    data = events_df[feature].dropna()
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3*IQR  # 3*IQR for extreme outliers
    upper_bound = Q3 + 3*IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_pct = len(outliers) / len(data) * 100
    
    axes[i].boxplot(data, vert=False)
    axes[i].set_xlabel(feature)
    axes[i].set_title(f'{feature} - {outlier_pct:.1f}% extreme outliers')

plt.tight_layout()

## Section 8: Summary & Recommendations

### 8.1 Feature Retention Justification

In [ ]:
if feature_importance is not None:
    # Cumulative importance
    feature_importance['cumulative_importance'] = feature_importance['importance'].cumsum() / feature_importance['importance'].sum()
    
    # Find features contributing to 90% of total importance
    n_features_90pct = (feature_importance['cumulative_importance'] <= 0.90).sum()
    
    print(f"Features needed for 90% cumulative importance: {n_features_90pct}/{len(feature_importance)}")
    print(f"Current feature set: {len([c for c in events_df.columns if c not in ['vehicle_id','start_time','end_time','pattern','y','is_train']])} features")

**Recommendation:**
> - **Core retention:** Top 20 features capture [X]% of predictive power
> - **Behavioral features** (pre-event context) show [Y]% lift in PR-AUC vs. baseline
> - **Vehicle normalization** reduces model bias by [Z]%
> - **Temporal features** justify night-shift monitoring priority

### 8.2 Feature Engineering Impact

In [ ]:
# Compare model performance with/without feature groups (requires re-training)
# This is conceptual - actual comparison would need separate training runs

impact_summary = pd.DataFrame({
    'Feature Group': ['Core Only', '+ Temporal', '+ Behavioral', '+ Vehicle-Normalized', 'All Features'],
    'Estimated PR-AUC': [0.65, 0.72, 0.78, 0.82, 0.85],  # Example values
    'Lift vs. Core': [0.00, 0.07, 0.13, 0.17, 0.20]
})

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(impact_summary['Feature Group'], impact_summary['Estimated PR-AUC'], color='steelblue')
ax.set_xlabel('PR-AUC')
ax.set_title('Feature Engineering Impact on Model Performance')
ax.set_xlim(0.5, 1.0)
plt.tight_layout()

print("\n" + impact_summary.to_string(index=False))

## Export Artifacts

In [ ]:
# Save feature statistics
feature_stats = events_df[numeric_features].describe().T
feature_stats.to_csv('../data/reports/feature_statistics.csv')

# Save correlation matrix
if feature_importance is not None and available_features:
    corr_matrix.to_csv('../data/reports/feature_correlations.csv')

print("✅ Feature analysis complete. Artifacts saved to data/reports/")